In [1]:
!pip install -q -U groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.5 MB/s eta 0:00:00


In [2]:
from groq import Groq
from google.colab import userdata
import textwrap

In [3]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
client = Groq(api_key=GROQ_API_KEY)

In [4]:
def summarized_articled_zero_shot(article_text):
  system_prompt = """
    You are a highly skilled AI assistant specializing in summarizing academic research papers.
    Your task is to generate a single, cohesive paragraph of no more than 150 words.
    The summary must cover these five key areas in order:
    1. Background/Introduction
    2. Research Objective
    3. Methodology
    4. Results Obtainted
    5. Key Takeaways/Conclusion.
    Ensure the output is a single, well-written paragraph.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [5]:
def summarized_articled_one_shot(article_text):
  system_prompt = """
    You are a highly skilled AI assistant specializing in summarizing academic research papers.
    Your task is to generate a single, cohesive paragraph of no more than 150 words.
    The summary must cover these five key areas in order:
    1. Background/Introduction
    2. Research Objective
    3. Methodology
    4. Results Obtainted
    5. Key Takeaways/Conclusion.
    Ensure the output is a single, well-written paragraph.

    Here is an example of how to perform the task:

    [EXAMPLE ARTICLE]
    Traditional lithium-ion batteries face limitations in energy density and safety. Solid-state batteries (SSBs) offer a promising alternative by using a solid electrolyte. This study aimed to develop a new polymer-based solid electrolyte with high ionic conductivity at room temperature. We synthesized a cross-linked polyethylene oxide (PEO) composite containing lithium bis(trifluoromethanesulfonyl)imide (LiTFSI) salt and ceramic nanoparticles (SiO2). The electrolyte's conductivity was measured using electrochemical impedance spectroscopy, and its stability was tested in a symmetric Li/Li cell. The resulting polymer composite electrolyte exhibited an ionic conductivity of 2.5 x 10^-4 S/cm at 25°C, a significant improvement over pure PEO. The Li/Li cell showed stable cycling for over 500 hours without short-circuiting. This research demonstrates that a nanoparticle-enhanced polymer composite is a viable approach for creating safe, high-conductivity solid electrolytes for next-generation batteries.

    [EXAMPLE SUMMARY]
    Amid the limitations of traditional lithium-ion batteries, this research addresses the need for safer, denser energy storage through solid-state batteries (SSBs). The primary objective was to develop a novel polymer-based solid electrolyte with high ionic conductivity at room temperature. The methodology involved synthesizing a cross-linked PEO composite with LiTFSI salt and SiO2 nanoparticles and testing its performance via electrochemical impedance spectroscopy. The key result was achieving a high ionic conductivity of 2.5 x 10^-4 S/cm at 25°C and demonstrating stable cell cycling for over 500 hours. The main takeaway is that nanoparticle-enhanced polymer composites present a promising and effective pathway for developing next-generation SSBs.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [6]:
def summarized_articled_few_shot(article_text):
  system_prompt = """
    You are a highly skilled AI assistant specializing in summarizing academic research papers.
    Your task is to generate a single, cohesive paragraph of no more than 150 words.
    The summary must cover these five key areas in order:
    1. Background/Introduction
    2. Research Objective
    3. Methodology
    4. Results Obtainted
    5. Key Takeaways/Conclusion.
    Ensure the output is a single, well-written paragraph.

    Here is an example of how to perform the task:

    Here are a few examples of how to perform the task correctly:

    ---
    [EXAMPLE 1 ARTICLE]
    Traditional lithium-ion batteries face limitations in energy density and safety. Solid-state batteries (SSBs) offer a promising alternative by using a solid electrolyte. This study aimed to develop a new polymer-based solid electrolyte with high ionic conductivity at room temperature. We synthesized a cross-linked polyethylene oxide (PEO) composite containing lithium bis(trifluoromethanesulfonyl)imide (LiTFSI) salt and ceramic nanoparticles (SiO2). The electrolyte's conductivity was measured using electrochemical impedance spectroscopy, and its stability was tested in a symmetric Li/Li cell. The resulting polymer composite electrolyte exhibited an ionic conductivity of 2.5 x 10^-4 S/cm at 25°C, a significant improvement over pure PEO. The Li/Li cell showed stable cycling for over 500 hours without short-circuiting. This research demonstrates that a nanoparticle-enhanced polymer composite is a viable approach for creating safe, high-conductivity solid electrolytes for next-generation batteries.

    [EXAMPLE 1 SUMMARY]
    Amid the limitations of traditional lithium-ion batteries, this research addresses the need for safer, denser energy storage through solid-state batteries (SSBs). The primary objective was to develop a novel polymer-based solid electrolyte with high ionic conductivity at room temperature. The methodology involved synthesizing a cross-linked PEO composite with LiTFSI salt and SiO2 nanoparticles and testing its performance via electrochemical impedance spectroscopy. The key result was achieving a high ionic conductivity of 2.5 x 10^-4 S/cm at 25°C and demonstrating stable cell cycling for over 500 hours. The main takeaway is that nanoparticle-enhanced polymer composites present a promising and effective pathway for developing next-generation SSBs.
    ---
    [EXAMPLE 2 ARTICLE]
    Urban green spaces are known to improve city environments, but their direct impact on mental health is less quantified. The goal was to measure the psychological effects of regular interaction with community gardens on urban residents. We conducted a 12-week study with 60 participants; 30 were assigned to spend at least two hours per week in a community garden, while the control group of 30 did not. Psychological well-being was assessed before and after the study using the Warwick-Edinburgh Mental Well-being Scale (WEMWBS). Participants in the gardening group showed a statistically significant increase in their WEMWBS scores, with an average improvement of 15%. The control group showed no significant change. This study concludes that active participation in community gardening provides measurable mental health benefits and should be encouraged as a public health initiative in urban planning.

    [EXAMPLE 2 SUMMARY]
    While the environmental benefits of urban green spaces are known, their specific mental health impact requires quantification. This study's objective was to measure the psychological benefits of community gardening on city dwellers. A 12-week trial was conducted where an experimental group engaged in gardening while a control group did not, with well-being assessed using the WEMWBS scale. The primary result was that the gardening group's well-being scores increased by an average of 15%, a statistically significant improvement not seen in the control group. The key takeaway is that community gardening offers tangible psychological benefits, supporting its inclusion in public health and urban planning strategies.
    ---"""
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [7]:
def summarized_articled_cot(article_text):
  system_prompt = """
    You are an expert academic summarizer. Your task is to summarize the provided article.
    First, think step-by-step. Silently, identify the following components from the text:
    1.  Background: What problem or context does the research address?
    2.  Objective: What was the main goal of the study?
    3.  Methodology: How was the research conducted?
    4.  Key Results: What were the most important findings?
    5.  Takeaways: What is the main conclusion or implication?

    After you have identified these parts, synthesize them into a single, cohesive paragraph of no more than 150 words.
    IMPORTANT: Do not show your step-by-step thinking in the final output. Only provide the final summary paragraph.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [8]:
def summarized_articled_tot(article_text):
  system_prompt = """
 You are a meticulous AI that explores multiple possibilities before finalizing an answer. To summarize the provided article, follow these steps internally:
    1.  Generate three distinct draft summaries. Each draft should approach the summary from a slightly different angle or with a different emphasis.
    2.  Critically evaluate the clarity, accuracy, and flow of each draft.
    3.  Select the best draft and refine it into a final, polished, single-paragraph summary of no more than 150 words.

    Your final output to the user should only be the single, best, refined summary. Do not show the drafts or your evaluation process.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [9]:
def summarized_articled_role_based(article_text):
  system_prompt = """
 Adopt the persona of a senior editor for the journal 'Nature'. You are writing a brief, compelling summary for the 'Research Highlights' section. Your summary must be accessible to a broad scientific audience, yet precise. It must be a single paragraph, under 150 words, and seamlessly integrate the background, objective, methodology, key findings, and conclusion of the provided paper.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [10]:
def summarized_articled_react(article_text):
  system_prompt = """
    You will summarize an article by following a strict Reason-Act cycle.
    Thought: The user wants a summary of the provided research article. I need to break the article down into its five core components (Background, Objective, Methods, Results, Conclusion), extract the key idea from each, and then synthesize them into a single, fluent paragraph under 150 words.

    Your final output should ONLY be the resulting summary paragraph, not your internal reasoning or action plan."""
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [31]:
def summarized_articled_direction_stimuli(article_text):
  system_prompt = """
    You are a skilled science writer. Summarize the following article in a single paragraph of no more than 150 words, covering the key aspects (background, objective, methods, results, takeaway).

    IMPORTANT: You MUST begin your summary with the exact phrase: 'In this article we read about,...'
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [34]:
def summarized_articled_step_back(article_text):
  system_prompt = """
    To summarize the provided article, you must first take a step back.
    1.  First, identify the abstract, high-level concept or principle the research is based on. What is the bigger picture?
    2.  Then, using this broad understanding as a frame, dive into the specifics of the article.
    3.  Finally, synthesize this information into a single, well-contextualized paragraph of no more than 150 words, covering the background, objective, methods, results, and conclusion.

    Only output the final summary paragraph.
    """
  user_prompt = f"Here is the article to summarize:\n\n---\n{article_text}"

  chat = client.chat.completions.create(messages=[
      {
      "role":"system",
      "content": system_prompt
      },
      {
      "role":"user",
      "content":user_prompt
      }
    ],
      model="llama3-8b-8192")
  summary = chat.choices[0].message.content

  return textwrap.fill(summary, width=80)

In [13]:
research_article = """
The Fallacy of Algorithmic Objectivity in Clinical Diagnostics

Author: Dr. Aris Thorne
Journal: Journal of Medical Ethics and Informatics
Year: 2024; Volume: 18, Issue: 2, pp. 112-114.
DOI: `10.1234/jmei.2024.0045`

Abstract
This commentary addresses the growing assumption that artificial intelligence (AI) and machine learning (ML) algorithms introduce inherent objectivity into clinical diagnostics, thereby eliminating human bias. I argue that this is a critical fallacy. Algorithms are products of human design and are trained on data derived from existing, often biased, clinical practices. This paper contends that without rigorous ethical oversight and "bias-auditing" protocols, diagnostic AI risks inheriting, amplifying, and ossifying the very biases it is claimed to solve. The concept of "algorithmic objectivity" should be replaced with a more cautious framework of "algorithm-assisted diligence."


The integration of artificial intelligence into medicine promises a revolution, particularly in the domain of diagnostics. Proponents champion machine learning models for their ability to parse vast datasets, identifying patterns that may elude the human eye. From radiology to histopathology, algorithms are being developed that can detect malignancies, predict disease progression, and recommend treatment pathways with increasing accuracy. A central claim in this optimistic narrative is that these systems offer a form of pure objectivity, a welcome antidote to the cognitive biases, fatigue, and environmental factors that can influence a human clinician's judgment. This perspective, however, is dangerously simplistic and rests on a fundamental fallacy.

The fallacy is the belief that an algorithm, by its mathematical nature, is inherently neutral. This ignores the simple truth that every algorithm is a product of its creators and, more importantly, its training data. Machine learning models for diagnostics are not developed in a vacuum; they are trained on immense libraries of historical medical data—images, lab results, and patient records collected over decades. This data is not an objective representation of biological reality. Rather, it is a record of past medical practices, complete with their embedded societal biases.

For instance, consider an algorithm trained to detect skin cancer from dermatological images. If the training dataset predominantly features images from fair-skinned individuals—a common issue in medical archives—the resulting algorithm will likely exhibit significantly lower accuracy when diagnosing malignancies in patients with darker skin. The algorithm is not "racist"; it is simply reflecting the structural biases of the data it was fed. It has learned to associate certain features with "normal" or "pathological" based on a skewed sample of reality. The outcome, however, is a racially disparate level of care, perpetuated under a veneer of computational neutrality.

This problem extends beyond data representation. Human-generated labels within these datasets are another vector for bias. A radiologist's interpretation of an ambiguous shadow on a chest X-ray is not pure signal; it is an expert judgment call. When millions of such judgment calls are used to train an algorithm, the model learns the "average" diagnostic patterns of the annotating physicians, including their collective risk aversion, common diagnostic misses, or regional idiosyncrasies in interpretation. The algorithm doesn't find a ground truth; it learns a consensus, warts and all.

Therefore, deploying diagnostic AI without a robust framework for ethical auditing is irresponsible. We must move beyond a naive faith in algorithmic objectivity. Instead, the focus should be on developing and implementing rigorous "bias-auditing" protocols. Before an algorithm is approved for clinical use, it must be tested across diverse demographic subgroups, including race, gender, and socioeconomic status. Its performance on these subgroups should be transparently reported. Any significant performance disparities must be addressed, either through dataset augmentation, algorithmic adjustments, or clear guidelines that warn clinicians of the tool's specific limitations.

In conclusion, AI in diagnostics is not an objective oracle but a powerful mirror that reflects the data, practices, and biases of the healthcare system it is born from. To treat it as anything else is to risk creating a new generation of medical tools that not only perpetuate existing health inequities but also entrench them with the unearned authority of a machine. The goal should not be to replace human judgment with supposed algorithmic objectivity, but to augment clinical practice with tools that are understood, scrutinized, and held to the highest ethical standards.

Keywords: Artificial Intelligence, Medical Ethics, Algorithmic Bias, Health Equity, Clinical Diagnostics, Machine Learning.

References:
1. Obermeyer, Z., Powers, B., Vogeli, C., & Mullainathan, S. (2019). Dissecting racial bias in an algorithm used to manage the health of populations. *Science*, 366(6464), 447-453.
2. Char, D. S., Shah, N. H., & Magnus, D. (2018). Implementing machine learning in health care—addressing ethical challenges. *The New England Journal of Medicine*, 378(11), 981-983."""

In [14]:
word_count = len(research_article.split())
print(f"The word count of the research article is: {word_count}")

The word count of the research article is: 753


In [15]:
print("------AI Generated Summary Zero Shot------")
summary_zero_shot = summarized_articled_zero_shot(research_article)
print(summary_zero_shot)

------AI Generated Summary Zero Shot------
Here is a summary of the article in a single paragraph:  The concept of
algorithmic objectivity in clinical diagnostics is a fallacy, as artificial
intelligence and machine learning algorithms are ultimately products of human
design and training data, which can be biased. An algorithm's neutrality is not
inherent, but rather a product of its creators and training data, which can
perpetuate existing societal biases. The assumption that algorithms can
eliminate human bias is simplistic and ignores the fact that human-generated
labels in training datasets can also introduce bias. Therefore, deploying
diagnostic AI without rigorous ethical oversight and "bias-auditing" protocols
is irresponsible. Instead, the focus should be on developing and implementing
protocols to test algorithms across diverse demographic subgroups, addressing
performance disparities, and implementing transparent reporting. By
acknowledging the flaws in algorithmic objectivit

In [16]:
summary_count = len(summary_zero_shot.split())
print(f"Word count of summary is: {summary_count}")

Word count of summary is: 155


In [17]:
print("------AI Generated Summary One Shot------")
summary_one_shot = summarized_articled_one_shot(research_article)
print(summary_one_shot)

------AI Generated Summary One Shot------
Here is a summarized paragraph covering the five key areas:  This commentary by
Dr. Aris Thorne challenges the assumption that artificial intelligence (AI) and
machine learning (ML) algorithms introduce inherent objectivity into clinical
diagnostics, thereby eliminating human bias. Instead, the author argues that
algorithms are products of human design and are trained on existing, often
biased, clinical practices. As a result, diagnostic AI risks inheriting,
amplifying, and ossifying the very biases it is claimed to solve. The concept of
"algorithmic objectivity" should be replaced with a more cautious framework of
"algorithm-assisted diligence." To address this issue, the author proposes the
development and implementation of rigorous "bias-auditing" protocols, including
testing across diverse demographic subgroups and transparent reporting of
performance disparities. By recognizing the limitations of AI in diagnostics and
focusing on ethical o

In [18]:
summary_count_one_shot = len(summary_one_shot.split())
print(f"Word count of summary is: {summary_count_one_shot}")

Word count of summary is: 149


In [19]:
print("------AI Generated Summary Few Shot------")
summary_few_shot = summarized_articled_few_shot(research_article)
print(summary_few_shot)

------AI Generated Summary Few Shot------
Here is a summary of the article in a single paragraph:  This commentary
addresses the misconception that artificial intelligence (AI) and machine
learning (ML) algorithms introduce objectivity into clinical diagnostics,
eliminating human bias. The author argues that this is a fallacy, as algorithms
are products of human design and are trained on data derived from existing,
often biased, clinical practices. This paper contends that without rigorous
ethical oversight and "bias-auditing" protocols, diagnostic AI risks inheriting,
amplifying, and ossifying the very biases it is claimed to solve. The concept of
"algorithmic objectivity" should be replaced with a more cautious framework of
"algorithm-assisted diligence." The author emphasizes the importance of testing
AI algorithms across diverse demographic subgroups, addressing any significant
performance disparities, and implementing robust "bias-auditing" protocols to
ensure fairness and equity 

In [20]:
summary_count_few_shot = len(summary_few_shot.split())
print(f"Word count of summary is: {summary_count_few_shot}")

Word count of summary is: 133


In [21]:
print("------AI Generated Summary Chain-of-thoughts------")
summary_cot = summarized_articled_cot(research_article)
print(summary_cot)

------AI Generated Summary Chain-of-thoughts------
Here is the summarized article:  The assumption that artificial intelligence and
machine learning algorithms introduce inherent objectivity into clinical
diagnostics is a critical fallacy. Algorithms are products of human design,
trained on data derived from existing clinical practices, which often contain
biases. Unless rigorous ethical oversight and "bias-auditing" protocols are
implemented, diagnostic AI risks inheriting and amplifying these biases. The
concept of "algorithmic objectivity" should be replaced with a more cautious
framework of "algorithm-assisted diligence." To ensure ethical use, AI in
diagnostics must be tested across diverse demographic subgroups and its
performance transparently reported, addressing any significant performance
disparities through dataset augmentation, algorithmic adjustments, or guidelines
that warn clinicians of the tool's limitations. By recognizing AI as a mirror
that reflects the data, practic

In [22]:
summary_count_cot = len(summary_cot.split())
print(f"Word count of summary is: {summary_count_cot}")

Word count of summary is: 150


In [24]:
print("------AI Generated Summary Total-of-thoughts------")
summary_tot = summarized_articled_tot(research_article)
print(summary_tot)

------AI Generated Summary Total-of-thoughts------
Here is a summary of the article:  The notion that artificial intelligence (AI)
and machine learning (ML) algorithms introduce inherent objectivity into
clinical diagnostics is a fallacy. These algorithms are products of human design
and are trained on data derived from existing, often biased, clinical practices.
Therefore, without rigorous ethical oversight and bias-auditing protocols,
diagnostic AI risks inheriting, amplifying, and ossifying the very biases it is
claimed to solve. Instead of relying on "algorithmic objectivity," the focus
should be on developing and implementing protocols to identify and mitigate
biases, ensuring that diagnostic tools are transparent and equitable in their
performance across various demographic subgroups.


In [25]:
summary_count_tot = len(summary_tot.split())
print(f"Word count of summary is: {summary_count_tot}")

Word count of summary is: 104


In [26]:
print("------AI Generated Summary Role-based------")
summary_role_based = summarized_articled_role_based(research_article)
print(summary_role_based)

------AI Generated Summary Role-based------
In a timely commentary, Dr. Aris Thorne challenges the assumption that
artificial intelligence (AI) and machine learning (ML) algorithms introduce
inherent objectivity into clinical diagnostics, effectively eliminating human
bias. Thorne argues that algorithms are products of human design and are trained
on data derived from existing, often biased, clinical practices, and thus risk
inheriting, amplifying, and ossifying the very biases they aim to solve. The
author contends that the concept of "algorithmic objectivity" should be replaced
with a more cautious framework of "algorithm-assisted diligence." To ensure
responsible deployment, Thorne advocates for rigorous "bias-auditing" protocols,
transparency in performance disparities across diverse subgroups, and clear
guidelines on algorithmic limitations. By acknowledging the limitations of AI
and promoting ethical oversight, we can harness machine learning to improve
clinical decision-making w

In [27]:
summary_count_role_based = len(summary_role_based.split())
print(f"Word count of summary is: {summary_count_role_based}")

Word count of summary is: 130


In [28]:
print("------AI Generated Summary React------")
summary_react = summarized_articled_react(research_article)
print(summary_react)

------AI Generated Summary React------
As artificial intelligence and machine learning are integrated into clinical
diagnostics, the assumption that these algorithms bring inherent objectivity has
become increasingly popular. However, this assumption is a critical fallacy. AI
algorithms are products of human design, trained on data derived from existing
and often biased clinical practices, and may inherit, amplify, and ossify these
biases. To avoid perpetuating biases, diagnostic AI requires rigorous ethical
oversight and bias-auditing protocols, which involve testing performance across
diverse demographics and transparently reporting results. By replacing the
concept of algorithmic objectivity with algorithm-assisted diligence, we can
enhance clinical practice and promote health equity.


In [29]:
summary_count_react = len(summary_react.split())
print(f"Word count of summary is: {summary_count_react}")

Word count of summary is: 99


In [32]:
print("------AI Generated Summary Direction-Stimulus------")
summary_direction_stimuli = summarized_articled_direction_stimuli(research_article)
print(summary_direction_stimuli)

------AI Generated Summary Direction-Stimulus------
In this article we read about the misconception that artificial intelligence
(AI) and machine learning (ML) algorithms introduce inherent objectivity into
clinical diagnostics, eliminating human bias. Dr. Aris Thorne argues that this
is a critical fallacy, as algorithms are products of human design and trained on
data derived from existing, often biased, clinical practices. Thorne contends
that without rigorous ethical oversight and "bias-auditing" protocols,
diagnostic AI risks inheriting, amplifying, and ossifying the very biases it is
claimed to solve. The article highlights that machine learning models are not
developed in a vacuum and are trained on historical medical data, which is often
biased and reflects the structural biases of the data. The author concludes that
deploying diagnostic AI without ethical auditing is irresponsible and that the
focus should be on developing and implementing rigorous "bias-auditing"
protocols to 

In [33]:
summary_count_direction_stimuli = len(summary_direction_stimuli.split())
print(f"Word count of summary is: {summary_count_direction_stimuli}")

Word count of summary is: 151


In [35]:
print("------AI Generated Summary Step-Back------")
summary_step_back = summarized_articled_step_back(research_article)
print(summary_step_back)

------AI Generated Summary Step-Back------
Here is the summarized paragraph:  This commentary highlights the fallacy of
assuming that artificial intelligence (AI) and machine learning (ML) algorithms
introduce inherent objectivity into clinical diagnostics, eliminating human
bias. Instead, algorithms are designed by humans and trained on data derived
from existing, often biased, clinical practices. This paper argues that without
rigorous ethical oversight and "bias-auditing" protocols, diagnostic AI risks
inheriting, amplifying, and ossifying the biases it is claimed to solve. It
proposes a more cautious framework of "algorithm-assisted diligence" and
emphasizes the need for testing algorithms across diverse demographic subgroups
to address performance disparities and ensure transparency.


In [36]:
summary_count_step_back = len(summary_step_back.split())
print(f"Word count of summary is: {summary_count_step_back}")

Word count of summary is: 99
